# Experimental: Method to calculate the delay days (=recovery period) per country

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import plotly as py
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import covsirphy as cs
cs.__version__

## Load data

In [ ]:
data_loader = cs.DataLoader(directory="../input")
jhu_data = data_loader.jhu()
oxcgrt_data = data_loader.oxcgrt()
# Description/citation
print(oxcgrt_data.citation)

## Estimate number of days of delay for a given country

In [ ]:
country = "Japan"
scn = cs.Scenario(country=country)
scn.register(jhu_data, extras=[oxcgrt_data])

indicators = oxcgrt_data.cleaned().columns.values[1:]
print("Indicators: ", indicators)

records = scn.records("CIFR", show_figure=False)
print("Targets: ", records.columns.values[1:])

In [ ]:
indicator, target = "Stringency_index", "Confirmed"
days, df_periods = scn.estimate_delay(oxcgrt_data, indicator=indicator, target=target)

In [ ]:
print("Days of delay for", country, ":", round(days, 2))

## View the calculated periods in a graph

In [ ]:
df_periods["Cumulative days"] = df_periods["Period Length"].cumsum()

plt.plot(records["Date"], records[target])
plt.xlabel("Date")
plt.ylabel(target)
plt.title(target + "over time with change periods")

start_date = records["Date"].values[0]
for day in df_periods["Cumulative days"].values[1:].astype(int):
    plt.axvline(start_date + np.timedelta64(day, 'D'), linestyle=":", color="black")

## Calculate delay days for every country

In [ ]:
countries = jhu_data.countries()
delay_days = {}
warnings.simplefilter("error", category=UserWarning)
for country in countries:
    scn = cs.Scenario(country=country)
    scn.register(jhu_data, extras=[oxcgrt_data])
    days, df_periods = scn.estimate_delay(indicator=indicator, target=target)
    delay_days[country] = days

In [ ]:
data = dict(
    type = 'choropleth',
    locations = list(delay_days.keys()),
    locationmode='country names',
    z=list(delay_days.values()),
    colorscale=px.colors.named_colorscales()[9],
    colorbar=dict(thickness=30, ticklen=7, tickfont=dict(size=20)),
)

map = go.Figure(
    data=[data],
    layout = go.Layout(
        title=f"Average number of delay days calculated with change in {target} and {indicator}",
        geo=dict(bgcolor= 'rgba(233,233,233,0.3)')))

py.offline.iplot(map)

In [ ]:
print(np.mean(list(delay_days.values())))